
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1z
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os
from glob import glob


# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

True

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

Mounted at /content/drive


In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

### Part 1: Object Detection

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''

def split_data(set_name):
  data_dirs = '{}/data/'.format(BASE_DIR)
 
  data_dirs_json = '{}/data/train.json'.format(BASE_DIR)    #data.train.json
  with open(data_dirs_json) as f:
        imgs_anns = json.load(f)

  input_path = os.path.join(data_dirs,set_name) #data/set_name
  dataset = {}

  for idx, v in enumerate(imgs_anns):
    # print(idx)
    # print(v)
    record = {}
    filename = os.path.join(input_path, v["file_name"])
    obj1 = {
        "bbox": v['bbox'],
        "bbox_mode": BoxMode.XYWH_ABS,
        "segmentation": v['segmentation'],
        "category_id": 0,
        "iscrowd" : v["iscrowd"]
    }

    obj2 = {
        "id" : v["id"],
        "category_id": v["category_id"],
        "iscrowd" : v["iscrowd"]
    }

    if not filename in dataset:
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      record["annotations"] = []
      record["segments_info"] = []
      dataset[filename] = record

    dataset[filename]['annotations'].append(obj1)
    dataset[filename]['segments_info'].append(obj2)
    # print(dataset[filename])

  # print(dataset)
  dataset = np.array(list(dataset.values()))
  np.random.shuffle(dataset)
  training, validation = dataset[:168], dataset[168:]

  test_data = []
  for filename in (glob(f'{BASE_DIR}/data/test/*')):
    record = {}
    height, width = cv2.imread(filename).shape[:2]
    record["file_name"] = filename
    record["height"] = height
    record["width"] = width
    record["annotations"] = []
    test_data.append(record)
  return training, validation, test_data


training_set, validation_set, test_set = split_data("train")

def get_detection_data(set_name):
  if set_name == "train":
    if type(training_set) is not list:
      return training_set.tolist()
    else:
      return training_set
  elif set_name == "test":
    return test_set
  else:
    if type(validation_set) is not list:
      return validation_set.tolist()
    else:
      return validation_set

In [ ]:
# train = get_detection_data("train")
# print(type(train))
# print(len(train))
# print(train[1])
# print(train[1]['annotations'][0])

In [ ]:
# val = get_detection_data("val")
# print(type(val))
# print(len(val))
# print(val[0])

In [ ]:
# test = get_detection_data("test")
# print(type(test))
# print(len(test))
# print(test[0])

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''

for d in ["train", "val","test"]:
    # print(d)
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["planes"])
planes_metadata = MetadataCatalog.get("data_detection_train")

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''

dataset_dicts = get_detection_data("train")
for d in random.sample(dataset_dicts, 3):
    # print(d)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=planes_metadata, scale=0.5)
    # print(d.values())
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000 #500   
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir drive/My\ Drive/CMPT_CV_lab3/output

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''

from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=planes_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''

evaluator = COCOEvaluator("data_detection_val", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "data_detection_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))



[11/07 00:04:47 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/07 00:04:47 d2.data.common]: Serializing 30 elements to byte tensors and concatenating them all ...
[11/07 00:04:47 d2.data.common]: Serialized dataset takes 3.10 MiB
[11/07 00:04:47 d2.evaluation.evaluator]: Start inference on 30 batches
[11/07 00:05:06 d2.evaluation.evaluator]: Inference done 11/30. Dataloading: 0.0039 s/iter. Inference: 1.2852 s/iter. Eval: 0.0004 s/iter. Total: 1.2895 s/iter. ETA=0:00:24
[11/07 00:05:12 d2.evaluation.evaluator]: Inference done 16/30. Dataloading: 0.0051 s/iter. Inference: 1.2218 s/iter. Eval: 0.0004 s/iter. Total: 1.2277 s/iter. ETA=0:00:17
[11/07 00:05:18 d2.evaluation.evaluator]: Inference done 20/30. Dataloading: 0.0044 s/iter. Inference: 1.3113 s/iter. Eval: 0.0004 s/iter. Total: 1.3165 s/iter. ETA=0:00:13
[11/07 00:05:24 d2.evaluation.evaluator]: Inference done 24

### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''


'\n# Bring any changes and updates regarding the improvement in here\n'

## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
training_set = get_detection_data("train")

In [ ]:

dicts = []

for index in training_set:
  filename = index['file_name']
  annos =  index['annotations']
  height_in =  index['height']
  width_in = index['width']

  objects = []
  img = cv2.imread(filename)

  for tot_segs in annos:
    bbox = tot_segs['bbox']
    x1 = bbox[0]
    y1 = bbox[1]
    x2 = x1+bbox[2]
    y2 = y1+bbox[3]
    segmentation = tot_segs['segmentation']
    
    mask = detectron2.utils.visualizer.GenericMask(segmentation, height_in,width_in).mask

    obj_mask = mask[int(y1):int(y2), int(x1):int(x2)]
    obj_mask = cv2.resize(obj_mask, (128, 128), interpolation = cv2.INTER_AREA) 
    cropped = img[int(y1):int(y2), int(x1):int(x2)]
    obj_img = cv2.resize(cropped, (128, 128), interpolation = cv2.INTER_AREA) 

    obj = {
        "mask": obj_mask,
        "image" : obj_img
    }
    objects.append(obj)
  inner_dict = {}
  inner_dict['filename'] = filename
  inner_dict['annos'] = objects
  dicts.append(inner_dict)


In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
def get_instance_sample(data, idx, img=None):
  for total in range(len(dicts)):
    if dicts[total]['filename'] == data['file_name']:
      obj_mask = dicts [total] ['annos'] [idx] ['mask']
      obj_img = dicts [total] ['annos'] [idx] ['image']
      return obj_img, obj_mask



In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    img , mask = get_instance_sample(data, idx[1])
    img, mask = self.numpy_to_tensor(img, mask)
    img = img.reshape((3,128,128))
    mask = mask.reshape((1,128,128))
    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
  
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        self.input_conv = conv(3, 16)
        self.down1 = down(16, 32)
        self.down2 = down(32, 64)
        self.down3 = down(64, 128)
        self.down4 = down(128, 256)
        self.down5 = down(256, 512)

        
        # Decoder
        self.up1 = up(512, 256)
        self.up2 = up(256, 128)
        self.up3 = up(128, 64)
        self.up4 = up(64, 32)
        self.up5 = up(32, 4)
        self.output_conv = conv(4, 1, False) # ReLu activation is removed to keep the logits for the loss function

    def forward(self, input):
      y = self.input_conv(input)
      y = self.down1(y)
      y = self.down2(y)
      y = self.down3(y)
      y = self.down4(y)
      y = self.down5(y)
      y = self.up1(y)
      y = self.up2(y)
      y = self.up3(y)
      y = self.up4(y)
      y = self.up5(y)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 50 #5
batch_size = 4
learning_rate = 0.001
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD
# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True).clone().detach()
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True).clone().detach()
    # print(dicts)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

total_iou = 0
total_images =0 
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cpu().detach()
    mask = torch.unsqueeze(mask,1)
    pred = model(img).cpu().detach()

    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    for i in range(img.shape[0]):
      predicted_image = pred[i]
      predicted_image = nn.Sigmoid()(predicted_image)[0]
      predicted_image = np.array(predicted_image)
      predicted_image = np.rint(predicted_image)

      masked = mask[i]
      ground_truth = nn.Sigmoid()(masked)[0]
      ground_truth = np.array(ground_truth)
      ground_truth = np.rint(ground_truth)

      Inter_o_union = np.sum(np.logical_and (ground_truth, predicted_image)) / np.sum(np.logical_or (ground_truth, predicted_image))
      total_images+=1
      total_iou += Inter_o_union


mean = total_iou/total_images

print("\n #images: {}, Mean IoU: {}".format(total_images,mean))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/837 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru


 #images: 6694, Mean IoU: 0.812196756916907


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''

loader, dataset = get_plane_dataset('train', 3)

img, mask = next(iter(loader))
img = img.cuda()
mask = mask.cpu().detach()
pred = model(img).cpu().detach()
for i in range(3):
  image = img[i].cpu()
  im = transforms.ToPILImage()(image)
  cv2_imshow(np.array(im))
  prediction = np.rint(np.array(nn.Sigmoid()(pred[i])[0]))*255
  masked = np.array(mask[i])[0]*255
  cv2_imshow(prediction)
  cv2_imshow(masked)


## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

def sigmoid_func(x):
  return 1/(1 + np.exp(-x))

def get_prediction_mask(data):
  # print(data)
  filename = data['file_name']
  height = data['height']
  width = data['width']
  annotations = data['annotations']
  img = cv2.imread(filename)

  prediction_mask = np.zeros([height, width])
  ground_truth_mask = np.zeros([height, width])

  if len(annotations) == 0:
    #test code
    bbox_finding = predictor(img)['instances']
    for i in range(len(bbox_finding)):
      bbox = np.array(np.floor(list(bbox_finding[i]._fields['pred_boxes'])[0].cpu().numpy()), dtype=np.uint32)
      x1 = bbox[0]
      y1 = bbox[1]
      x2 = bbox[2]
      y2 = bbox[3]

      pred = img[int(y1):int(y2), int(x1):int(x2)]
      pred = cv2.resize(pred, (128, 128), interpolation = cv2.INTER_AREA)
      pred = torch.unsqueeze(torch.tensor(transforms.ToTensor()(pred), device=torch.device('cuda')),0)
      
      pred = model(pred).cpu().detach().numpy()[0]
      pred = cv2.resize(pred[0], (int(x2)-int(x1), int(y2)-int(y1)), interpolation = cv2.INTER_AREA)
      pred = sigmoid_func(pred)
      pred = np.array(pred)
      pred = np.rint(pred)
      pred = pred*(i+1)

    
      mask = prediction_mask[int(y1):int(y2), int(x1):int(x2)] 
      mask[mask == 0] = 10000

      covering_pm = np.minimum(mask, pred)
      covering_pm[covering_pm == 1000] = 0
      prediction_mask[int(y1):int(y2), int(x1):int(x2)] = covering_pm

    for i,j in enumerate(annotations):
      bbox = anno['bbox']
      x1 = bbox[0]
      y1 = bbox[1]
      x2 = x1+bbox[2]
      y2 = y1+bbox[3]

      real_mask = detectron2.utils.visualizer.GenericMask(anno['segmentation'], height, width).mask
      covering_tm = np.maximum(ground_truth_mask[int(y1):int(y2), int(x1):int(x2)],real_mask[int(y1):int(y2), int(x1):int(x2)]*(idx+1))
      ground_truth_mask[int(y1):int(y2), int(x1):int(x2)] = covering_tm


    
  else:
    #train code
    for idx, anno in enumerate(annotations):
      bbox = anno['bbox']
      x1 = bbox[0]
      y1 = bbox[1]
      x2 = x1+bbox[2]
      y2 = y1+bbox[3]

      pred = img[int(y1):int(y2), int(x1):int(x2)]
      pred = cv2.resize(pred, (128, 128), interpolation = cv2.INTER_AREA)
      pred = torch.unsqueeze(torch.tensor(transforms.ToTensor()(pred), device=torch.device('cuda')),0)
      
      pred = model(pred).cpu().detach().numpy()[0]
      pred = cv2.resize(pred[0], (int(x2-x1), int(y2-y1)), interpolation = cv2.INTER_AREA)
      pred = sigmoid_func(pred)
      pred = np.array(pred)
      pred = np.rint(pred)
      pred = pred*(idx+1)

    
      mask = prediction_mask[int(y1):int(y2), int(x1):int(x2)] 
      mask[mask == 0] = 10000

      covering_pm = np.minimum(mask, pred)
      covering_pm[covering_pm == 1000] = 0
      prediction_mask[int(y1):int(y2), int(x1):int(x2)] = covering_pm
      


      real_mask = detectron2.utils.visualizer.GenericMask(anno['segmentation'], height, width).mask
      covering_tm = np.maximum(ground_truth_mask[int(y1):int(y2), int(x1):int(x2)],real_mask[int(y1):int(y2), int(x1):int(x2)]*(idx+1))
      ground_truth_mask[int(y1):int(y2), int(x1):int(x2)] = covering_tm

  gt_mask = ground_truth_mask
  gt_mask = torch.tensor(gt_mask,device=torch.device('cuda'))
  pred_mask = prediction_mask
  pred_mask = torch.tensor(prediction_mask, device=torch.device('cuda'))

  return img, gt_mask, pred_mask # gt_mask could be all zero when the ground truth is not given.
  

### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''

training_set = get_detection_data('train')
# length = len(training_set)
random = np.random.randint(0,167,3)
for i in random:

  img, gt_mask, pred_mask = get_prediction_mask(training_set[i])
  pred_mask = pred_mask.cpu().numpy()
  value_pred = pred_mask.max()
  value_pred = 255./value_pred
  pred_mask = pred_mask * value_pred

  gt_mask = gt_mask.cpu().numpy()
  value_gt = gt_mask.max()
  value_gt = 255./value_gt
  gt_mask = gt_mask * value_gt

  cv2_imshow(cv2.resize(img, (img.shape[1], img.shape[0]), interpolation = cv2.INTER_AREA))
  cv2_imshow(cv2.resize(gt_mask, (gt_mask.shape[1], gt_mask.shape[0]), interpolation = cv2.INTER_AREA))
  cv2_imshow(cv2.resize(pred_mask, (pred_mask.shape[1], pred_mask.shape[0]), interpolation = cv2.INTER_AREA))



In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)


'''
# Writing the predictions of the validation set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('val'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)



'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


  0%|          | 0/168 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
DatasetCatalog.remove('data_detection_train')
DatasetCatalog.remove('data_detection_test')
DatasetCatalog.remove('data_detection_val')

for d in ["train", "val","test"]:
    # print(d)
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["planes"])
planes_metadata = MetadataCatalog.get("data_detection_train")

### Network

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500 #500   
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

### Training

In [ ]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

### Evaluation and Visualization

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''

from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("val")
# print(len(dataset_dicts))
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=planes_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''

evaluator = COCOEvaluator("data_detection_val", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "data_detection_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[11/07 05:53:44 d2.evaluation.coco_evaluation]: Trying to convert 'data_detection_val' to COCO format ...
[11/07 05:53:44 d2.data.datasets.coco]: Converting annotations of dataset 'data_detection_val' to COCO format ...)
[11/07 05:53:44 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[11/07 05:53:44 d2.data.datasets.coco]: Conversion finished, #images: 30, #annotations: 1882
[11/07 05:53:44 d2.data.datasets.coco]: Caching COCO format annotations at '/content/drive/My Drive/CMPT_CV_lab3/output/data_detection_val_coco_format.json' ...
[11/07 05:53:46 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   planes   | 1882         |
|            |              |
[11/07 05:53:46 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/07 05:53:46 d2.data.common]: Serializing 30 elements to by